# TOPIC 07: SQL DATABASES

- onl01-dtsc-ft-022221
- 03/05/21

## LEARNING OBJECTIVES:

- Understand what a relational database is and how it is different than a DataFrame/Excel sheet.
- Understand how to read database map (AKA "Entity Relationship Diagram (ERD)")
    - Primary keys vs forgein keys
- Understand how to select, filter, order, and group data using SQL
- Understand the different types of Joins 
#### 🏝 Breakout Group Activity:  [Survive on sql-island](https://sql-island.informatik.uni-kl.de/)

### Questions:

- When doing some of the admin tasks like inserting rows, I tried to run an insert command again and got a "database is locked" error. In what circumstances does this happen, and are there best practices for closing the connection to avoid locking?


- The SQL quiz had a question about the role of Associated Entities on many-to-many joins. I went back through the lessons and couldn't find a section which talked about that term. What should we know about many-to-many joins other than that they can become very large?



- Can you rephrase the below statement? 
>"For example, let's say you have another table 'restaurants' that has many columns including name, city, and rating. If you were to join this 'restaurants' table with the offices table using the shared city column, you might get some unexpected behavior. That is, in the office table, there is only one office per city. However, because there will likely be more than one restaurant for each of these cities in your second table, you will get unique combinations of Offices and Restaurants from your join. If there are 513 restaurants for Boston in your restaurant table and 1 office for Boston, your joined table will have each of these 513 rows, one for each restaurant along with the one office.<br>
If you had 2 offices for Boston and 513 restaurants, your join would have 1026 rows for Boston; 513 for each restaurant along with the first office and 513 for each restaurant with the second office. Three offices in Boston would similarly produce 1539 rows; one for each unique combination of restaurants and offices. This is where you should be particularly careful of many to many joins as the resulting set size can explode drastically potentially consuming vast amounts of memory and other resources."


# SQL Databases

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png">


- SQL is designed to work with **relational data**. 
- This really just means pieces of data that are **related to eachother**.

- Each table has a **primary key** (like a DataFrame index), with a unique index for each row in the database.
- The name of the primary key is preceded by an asterix (\*). 

- Columns that are the **primary key one on table** can also appear on **other tables**. 
    - Then it is refered to as a **foreign key** aka the primary key from a different ("foreign") table. 

### ⨠ Q: Why do we need databases? Why can't we just use a bunch of Pandas DataFrames?

- 

## Querying Databases - `SELECT`ing data



- To retrieve data from one or more tables you usually use a `SELECT` statement. 
```SQL
SELECT * FROM table;
```


> - NOTE: SQL queries dot not _have_ to be all-caps, but it is a convention to help differentiate sql syntax versus names of tables/columns.



- A more advanced select query.
```SQL
SELECT col1, col2, col3
FROM table
WHERE records match criteria
LIMIT 100;
```

- **All select statements must:**
    1. **start with the `SELECT`**
    2. followed by **what you want to select**. Separate multiple column names separated by a `,` 
    3. Then specify where the data is coming `FROM` followed by the table name. 
    4. **Afterward, you can provide conditions such as filters or sorting**.

```SQL
SELECT *
FROM payments
ORDER BY amount DESC
LIMIT 10;
```

<!-- 
```SQL

```
 -->


## SQL with `sqlite3`

- Use `sqlite3` for SQL queries in Python.
    1. Connect to database
    2. Create a cursor.
    3. Form your query
    4. Execute/fetch your results.

```python
import sqlite3
connection = sqlite3.connect('pet_database.db') # Creates pet_database, but empty until create a table    
cursor = connection.cursor()


# Select from table
cursor.execute('''SELECT name FROM cats;''').fetchall()

```

In [2]:
import pandas as pd
import os
os.getcwd()

'/Users/jamesirving/Documents/GitHub/_COHORT_NOTES/022221FT/Online-DS-FT-022221-Cohort-Notes/Phase_1/topic_07_SQL_and_relational_databases'

In [3]:
## Our data.sqlite is in the datasets folder in our note repo.
db = '../../datasets/SQL/data.sqlite'

In [4]:
import sqlite3
# connect to database
conn = sqlite3.connect(db)
cur = conn.cursor()

type(conn)

sqlite3.Connection

### How to get all of the table names in a database
>- The container for all tables in a database with sqlite3 is called `sqlite_master` 
>- We can find the name of all of the tables in a db using: 
>```python
table_names = cur.execute("""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""").fetchall()
```

In [5]:
# Get table names
table_names = cur.execute("""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""").fetchall()
table_names

[('orderdetails',),
 ('payments',),
 ('offices',),
 ('customers',),
 ('orders',),
 ('productlines',),
 ('products',),
 ('employees',),
 ('cats',)]

In [6]:
## Clean up the list of table names to be just 1 list with strings
table_names = [x[0] for x in table_names]
table_names

['orderdetails',
 'payments',
 'offices',
 'customers',
 'orders',
 'productlines',
 'products',
 'employees',
 'cats']

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png" width=500>

### How to get the column names after executing a query:


In [7]:
df = pd.DataFrame(cur.execute("select * from products").fetchall())
df

,0,1,2,3,4,5,6,7,8
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00
...,...,...,...,...,...,...,...,...,...
105,S700_3505,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17
106,S700_3962,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31
107,S700_4002,American Airlines: MD-11S,Planes,1:700,Second Gear Diecast,Polished finish. Exact replia with official lo...,8820,36.27,74.03
108,S72_1253,Boeing X-32A JSF,Planes,1:72,Motor City Art Classics,"10"" Wingspan with retractable landing gears.Co...",4857,32.77,49.66


> - the cursor has a `.description` that contains information about the column names

In [8]:
cur.description

(('productCode', None, None, None, None, None, None),
 ('productName', None, None, None, None, None, None),
 ('productLine', None, None, None, None, None, None),
 ('productScale', None, None, None, None, None, None),
 ('productVendor', None, None, None, None, None, None),
 ('productDescription', None, None, None, None, None, None),
 ('quantityInStock', None, None, None, None, None, None),
 ('buyPrice', None, None, None, None, None, None),
 ('MSRP', None, None, None, None, None, None))

In [9]:
## Get a list of just the column names from the description
col_names =[col[0] for col in cur.description]
print(col_names)

['productCode', 'productName', 'productLine', 'productScale', 'productVendor', 'productDescription', 'quantityInStock', 'buyPrice', 'MSRP']


In [10]:
df = pd.DataFrame(cur.execute('select * from products').fetchall(),
                  columns=col_names)
df.head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [11]:
cur.execute("""PRAGMA table_info(products)""").fetchall()

[(0, 'productCode', '', 0, None, 0),
 (1, 'productName', '', 0, None, 0),
 (2, 'productLine', '', 0, None, 0),
 (3, 'productScale', '', 0, None, 0),
 (4, 'productVendor', '', 0, None, 0),
 (5, 'productDescription', '', 0, None, 0),
 (6, 'quantityInStock', '', 0, None, 0),
 (7, 'buyPrice', '', 0, None, 0),
 (8, 'MSRP', '', 0, None, 0)]

>### Quick Activity: Make executing the query -> dataframe into a function

In [15]:
def query_to_df(query,cursor=cur):
    df = pd.DataFrame(cursor.execute(query).fetchall(),
                  columns=[col[0] for col in cursor.description])
    return df

In [16]:
query_to_df("""SELECT * FROM EMPLOYEES""")

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


# FILTERING AND ORDERING

- `ORDER BY` - `DESC`/`ASC`
- `LIMIT`
- `BETWEEN`
- `NULL`
- `COUNT`
- `GROUP BY`

In [17]:
query = """select * from products
GROUP BY productLine 
ORDER BY quantityInStock DESC;"""
query_to_df(query)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S18_1342,1937 Lincoln Berline,Vintage Cars,1:18,Motor City Art Classics,"Features opening engine cover, doors, trunk, a...",8693,60.62,102.74
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
2,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
3,S18_3259,Collectable Wooden Train,Trains,1:18,Carousel DieCast Legends,Hand crafted wooden toy train set is in about ...,6450,67.56,100.84
4,S18_1662,1980s Black Hawk Helicopter,Planes,1:18,Red Start Diecast,1:18 scale replica of actual Army's UH-60L BLA...,5330,77.27,157.69
5,S18_3029,1999 Yamaha Speed Boat,Ships,1:18,Min Lin Diecast,Exact replica. Wood and Metal. Many extras inc...,4259,51.61,86.02
6,S12_1666,1958 Setra Bus,Trucks and Buses,1:12,Welly Diecast Productions,"Model features 30 windows, skylights & glare r...",1579,77.90,136.67


## GROUPING DATA WITH SQL

- Like we do with Pandas, we can use GROUP BY statements in SQL and then apply **aggregate functions:**
    - `COUNT`
    - `MAX`
    - `MIN`
    - `SUM`
    - `AVG`

```SQL
SELECT city, COUNT(employeeNumber)
FROM offices 
JOIN employees
USING(officeCode)
GROUP BY city
ORDER BY count(employeeNumber) DESC;
```

In [19]:
query="""SELECT city, COUNT(employeeNumber)
FROM offices 
JOIN employees
USING(officeCode)
GROUP BY city
ORDER BY count(employeeNumber) DESC;"""
query_to_df(query)

,city,COUNT(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Tokyo,2
4,NYC,2
5,London,2
6,Boston,2


## ALIASING

- can assign a temporary name to data being imported
- Useful for `JOIN`,`GROUP BY`, and aggregates.

In [20]:
query="""SELECT city, COUNT(employeeNumber) AS numEmployees
               FROM offices
               JOIN employees
               USING(officeCode)
               GROUP BY 1
               ORDER BY numEmployees DESC;"""
query_to_df(query)

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Tokyo,2
4,NYC,2
5,London,2
6,Boston,2


In [21]:
query_to_df("""SELECT customerName,
               COUNT(customerName) AS number_purchases,
               MIN(amount) AS min_purchase,
               MAX(amount) AS max_purchase,
               AVG(amount) AS avg_purchase,
               SUM(amount) AS total_spent
               FROM customers
               JOIN payments
               USING(customerNumber)
               GROUP BY 1
               ORDER BY SUM(amount) DESC;""")

,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03
...,...,...,...,...,...,...
93,Royale Belge,4,1128.20,1627.56,7304.295000,29217.18
94,Frau da Collezione,2,17746.26,7612.06,12679.160000,25358.32
95,Atelier graphique,3,14571.44,6066.78,7438.120000,22314.36
96,Auto-Moto Classics Inc.,3,5858.56,9658.74,7184.753333,21554.26


## The `WHERE` Clause

In general, the `WHERE` clause filters query results by some condition. As you are starting to see, you can also combine multiple conditions.

- 
```python
cur.execute("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df
```


- To refine your searches, you can add `ORDER BY` and `LIMIT` clauses. 
    - The order by clause allows you to sort the results by a particular feature.
- Finally, the limit clause is typically the last argument in a SQL query and simply limits the output to a set number of results.

In [22]:
query_to_df("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",,Madrid,,28034,Spain,1370,227600.00
1,237,ANG Resellers,Camino,Alejandra,(91) 745 6555,"Gran Vía, 1",,Madrid,,28001,Spain,,0.00
2,344,CAF Imports,Fernandez,Jesus,+34 913 728 555,Merchants House,27-30 Merchant's Quay,Madrid,,28023,Spain,1702,59600.00
3,362,Gifts4AllAges.com,Yoshido,Juri,6175559555,8616 Spinnaker Dr.,,Boston,MA,51003,USA,1216,41900.00
4,458,"Corrida Auto Replicas, Ltd",Sommer,Martín,(91) 555 22 82,"C/ Araquil, 67",,Madrid,,28023,Spain,1702,104600.00
5,465,"Anton Designs, Ltd.",Anton,Carmen,+34 913 728555,"c/ Gobelas, 19-1 Urb. La Florida",,Madrid,,28023,Spain,,0.00
6,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100.00


## The `HAVING` clause

 The `HAVING` clause works similarly to the `WHERE` clause, except it is used to filter data selections on conditions **after** the `GROUP BY` clause.

In [25]:
query_to_df("""SELECT city, COUNT(customerNumber) AS number_customers
               FROM customers
               GROUP BY 1
               HAVING COUNT(customerNumber)>=5;""")

,city,number_customers
0,Madrid,5
1,NYC,5


## Combining `WHERE` and `HAVING`

We can also use the `WHERE` and `HAVING` clauses in conjunction with each other for more complex rules.

- For example, let's say we want a list of customers who have made at least 3 purchases of over 50K each.

In [27]:
query_to_df("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY 1
               HAVING count(amount) >= 3
               ORDER BY number_purchases_over_50K DESC;""")

,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Tokyo Collectables, Ltd",4
3,Technics Stores Inc.,4
4,Royale Belge,4
5,Reims Collectables,4
6,Muscle Machine Inc,4
7,"Dragon Souveniers, Ltd.",4
8,"Down Under Souveniers, Inc",4
9,Danish Wholesale Imports,4


In [ ]:
query_to_df("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY 1
               HAVING number_purchases_over_50K >= 3
               ORDER BY number_purchases_over_50K DESC;""")


# JOINS

### TYPES OF JOINS

- Joins may be:
    - INNER (default)
    - OUTER
    - LEFT 
    - RIGHT

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-join-statements-online-ds-sp-000/master/images/venn.png">


## `JOIN` Statements

### Primary vs Foreign Keys
- primary key:
- forgein key:


<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png">

### The `USING` clause

- If the column name is identical,you can use  is the `USING` clause. 
- Rather then saying on `tableA.column = tableB.column` we can simply say `using(column)`. 
- Only works if the column is **identically named** for both tables.

#### Task: Displaying product details along with order details

Let's say you need to generate some report that includes details about products from orders. To do that, we would need to take data from multiple tables in a single statement. 

In [28]:
query_to_df("""SELECT * 
               FROM orderdetails
               JOIN products
               ON orderdetails.productCode = products.productCode
               LIMIT 10;
               """)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,S18_1749,30,136.00,3,S18_1749,1917 Grand Touring Sedan,Vintage Cars,1:18,Welly Diecast Productions,This 1:18 scale replica of the 1917 Grand Tour...,2724,86.70,170.00
1,10100,S18_2248,50,55.09,2,S18_2248,1911 Ford Town Car,Vintage Cars,1:18,Motor City Art Classics,"Features opening hood, opening doors, opening ...",540,33.30,60.54
2,10100,S18_4409,22,75.46,4,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,1:18,Exoto Designs,This 1:18 scale precision die cast replica fea...,6553,43.26,92.03
3,10100,S24_3969,49,35.29,1,S24_3969,1936 Mercedes Benz 500k Roadster,Vintage Cars,1:24,Red Start Diecast,This model features grille-mounted chrome horn...,2081,21.75,41.03
4,10101,S18_2325,25,108.06,4,S18_2325,1932 Model A Ford J-Coupe,Vintage Cars,1:18,Autoart Studio Design,This model features grille-mounted chrome horn...,9354,58.48,127.13
5,10101,S18_2795,26,167.06,1,S18_2795,1928 Mercedes-Benz SSK,Vintage Cars,1:18,Gearbox Collectibles,This 1:18 replica features grille-mounted chro...,548,72.56,168.75
6,10101,S24_1937,45,32.53,3,S24_1937,1939 Chevrolet Deluxe Coupe,Vintage Cars,1:24,Motor City Art Classics,This 1:24 scale die-cast replica of the 1939 C...,7332,22.57,33.19
7,10101,S24_2022,46,44.35,2,S24_2022,1938 Cadillac V-16 Presidential Limousine,Vintage Cars,1:24,Classic Metal Creations,This 1:24 scale precision die cast replica of ...,2847,20.61,44.80
8,10102,S18_1342,39,95.55,2,S18_1342,1937 Lincoln Berline,Vintage Cars,1:18,Motor City Art Classics,"Features opening engine cover, doors, trunk, a...",8693,60.62,102.74
9,10102,S18_1367,41,43.13,1,S18_1367,1936 Mercedes-Benz 500K Special Roadster,Vintage Cars,1:18,Studio M Art Models,This 1:18 scale replica is constructed of heav...,8635,24.26,53.91


## Types of Relationships Between Tables

#### One-to-One, One-to-many, many-to-many Joins


- **Let's say we have databases A and B**


- **One-to-One joins:**
    - There is only 1 entry in database B that aligns with each individual entry in database A
    - e.g. A person and their social security number.
    
    
- **One-to-Many join:**
    - There are multiple entries in database B that match the entry in database A
    - e.g. Joining an order number from db A with the individual products in db B.
    
    
- **Many-to-many joins:**
    - There are multiple entries in database A that match multiple entries in database B.
    - e.g. A = classes at a college, B = students.

### One-to-One

- There is only 1 entry in database B that aligns with each individual entry in database A
- e.g. A person and their social security number.

In [29]:
## Preview full offices table
query_to_df("Select * FROM offices;")

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA


In [30]:
## Preview full employees table
query_to_df("Select * FROM employees;")

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


In [38]:
## One to one join
query_to_df('''SELECT * FROM employees 
JOIN offices USING(officeCode)
ORDER  BY officeCode;''')

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
3,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA),San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
4,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
5,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
6,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
7,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
8,1286,Tseng,Foon Yue,x2248,ftseng@classicmodelcars.com,3,1143,Sales Rep,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
9,1323,Vanauf,George,x4102,gvanauf@classicmodelcars.com,3,1143,Sales Rep,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA


### One-to-Many

- There are multiple entries in database B that match the entry in database A
- e.g. Joining an order number from db A with the individual products in db B.

In [32]:
## preview table 1 
query_to_df("SELECT * FROM products;")

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00
...,...,...,...,...,...,...,...,...,...
105,S700_3505,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17
106,S700_3962,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31
107,S700_4002,American Airlines: MD-11S,Planes,1:700,Second Gear Diecast,Polished finish. Exact replia with official lo...,8820,36.27,74.03
108,S72_1253,Boeing X-32A JSF,Planes,1:72,Motor City Art Classics,"10"" Wingspan with retractable landing gears.Co...",4857,32.77,49.66


In [34]:
## preview table 2
query_to_df("SELECT * FROM productlines;")

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,,
1,Motorcycles,Our motorcycles are state of the art replicas ...,,
2,Planes,"Unique, diecast airplane and helicopter replic...",,
3,Ships,The perfect holiday or anniversary gift for ex...,,
4,Trains,Model trains are a rewarding hobby for enthusi...,,
5,Trucks and Buses,The Truck and Bus models are realistic replica...,,
6,Vintage Cars,Our Vintage Car models realistically portray a...,,


In [36]:
## One-to-Many Join
query_to_df("""SELECT * 
               FROM products
               LEFT JOIN productlines
               USING(productLine);""")

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,textDescription,htmlDescription,image
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,Our motorcycles are state of the art replicas ...,,
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30,Attention car enthusiasts: Make your wildest c...,,
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94,Our motorcycles are state of the art replicas ...,,
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66,Our motorcycles are state of the art replicas ...,,
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00,Attention car enthusiasts: Make your wildest c...,,
...,...,...,...,...,...,...,...,...,...,...,...,...
105,S700_3505,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17,The perfect holiday or anniversary gift for ex...,,
106,S700_3962,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31,The perfect holiday or anniversary gift for ex...,,
107,S700_4002,American Airlines: MD-11S,Planes,1:700,Second Gear Diecast,Polished finish. Exact replia with official lo...,8820,36.27,74.03,"Unique, diecast airplane and helicopter replic...",,
108,S72_1253,Boeing X-32A JSF,Planes,1:72,Motor City Art Classics,"10"" Wingspan with retractable landing gears.Co...",4857,32.77,49.66,"Unique, diecast airplane and helicopter replic...",,


### Many-to-Many

- There are multiple entries in database A that match multiple entries in database B.
- e.g. A = classes at a college, B = students.

In [39]:
query_to_df("""SELECT * FROM offices
                        JOIN customers
                        USING(state);""")

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory,customerNumber,...,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,124,...,Nelson,Susan,4155551450,5677 Strong St.,,San Rafael,97562,USA,1165,210500.00
1,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,129,...,Murphy,Julie,6505555787,5557 North Pendale Street,,San Francisco,94217,USA,1165,64600.00
2,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,161,...,Hashimoto,Juri,6505556809,9408 Furth Circle,,Burlingame,94217,USA,1165,84600.00
3,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,205,...,Young,Julie,6265557265,78934 Hillside Dr.,,Pasadena,90003,USA,1166,90700.00
4,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,219,...,Young,Mary,3105552373,4097 Douglas Av.,,Glendale,92561,USA,1166,11000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,480,...,Semenov,Alexander,+7 812 293 0521,2 Pobedy Square,,Saint Petersburg,196143,Russia,,0.00
241,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,481,...,"Altagar,G M",Raanan,+ 972 9 959 8555,3 Hagalim Blv.,,Herzlia,47625,Israel,,0.00
242,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,484,...,Roel,José Pedro,(95) 555 82 82,"C/ Romero, 33",,Sevilla,41101,Spain,1702,65700.00
243,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,489,...,Smith,Thomas,(171) 555-7555,120 Hanover Sq.,,London,WA1 1DP,UK,1501,43300.00


## SQL Subqueries

- WHen making a subq, start by testing the  sub-Q itself before using in the larger query

```python
cur.execute("""SELECT lastName, firstName, officeCode
               FROM employees
               WHERE officeCode IN (SELECT officeCode
                                    FROM offices 
                                    WHERE country = "USA");
                                    """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df
```

###  Example Subquery 

```SQL
SELECT LAB_MONTH,MPI
,CW_UPI
,LAST_NAME
,FIRST_NAME
,FACILITY_NUMBER
,CCN
,FACILITY_NAME
,NUMBER_OF_CLINICS
,HOME_FACILITY
,TRANSIENT
,CASE WHEN ADMITDATE <= '7/1/2020' 
  AND (PATIENTDISCHARGEDATE >= '7/31/2020'
    OR PATIENTDISCHARGEDATE IS NULL)
  THEN 'FULL MONTH' ELSE 'LESS THAN A MONTH' END FULL_MONTH
,ADMITDATE AS ADMIT_DATE
,ADMITREASON AS ADMIT_REASON
,CASE WHEN PATIENTDISCHARGEDATE >= ADMITDATE
  THEN PATIENTDISCHARGEDATE ELSE NULL END DISCHARGE_DATE
FROM (
  SELECT *
  ,ROW_NUMBER() OVER (PARTITION BY MPI, FACILITY_NUMBER ORDER BY ADMITDATE DESC) ADMIT_ROW_NUM
  FROM (
    SELECT DISTINCT
    LAB_MONTH
    ,labs.IDPATIENT AS MPI
    ,map.CWIDPATIENT AS CW_UPI
    ,name.LASTNAME AS LAST_NAME
    ,name.FIRSTNAME AS FIRST_NAME
    ,cln.IDCORPCLINIC AS FACILITY_NUMBER
    ,TRANSLATE(MEDICARECERTNUM, '-', '') AS CCN
    ,NAMECLINIC AS FACILITY_NAME
    ,NUM_CLINICS AS NUMBER_OF_CLINICS
    ,CASE WHEN labs.IDCLINICNUMBER = pnt.HOME_LOCATIONID
      THEN 'YES' ELSE 'NO' END HOME_FACILITY
    ,per.TRANSIENT
    ,per.ADMITDATE
    ,per.ADMITREASON
    ,pnt.PATIENTDISCHARGEDATE
    ,pnt.REC_EFFECTIVE_START_DATE AS PNT_REC_EFFECTIVE_START_DATE
    FROM(
      SELECT *
      ,COUNT(DISTINCT IDCLINICNUMBER) OVER (PARTITION BY IDPATIENT) AS NUM_CLINICS
      ,TO_CHAR(LAST_DAY(DATEDRAW),'yyyymm') AS LAB_MONTH
      FROM AUD_RSLTS_VW
      WHERE AUD_CURRENT_ROW_FLAG = 'Y'
      AND DATEDRAW BETWEEN '2020-07-01' AND '2020-07-31'
      ) labs
    LEFT JOIN AUD_CLINIC_VW cln
    ON labs.IDCLINICNUMBER = cln.CODECLINIC
    LEFT JOIN CWPATIENTMAP_VW map
    ON labs.IDPATIENT = map.IDPATIENT
    LEFT JOIN (
      SELECT
      PATIENTID
      ,LASTNAME
      ,FIRSTNAME
      FROM MSTR_PATIENT_DIM_VW
      WHERE REC_CURRENT_IND = 1
      ) name
    ON labs.IDPATIENT = name.PATIENTID
    LEFT JOIN MSTR_PATIENT_DIM_VW pnt
    ON labs.IDPATIENT = pnt.PATIENTID
    AND labs.IDCLINICNUMBER = pnt.HOME_LOCATIONID
    LEFT JOIN MSTR_FACILITY_DIM_VW fac
    ON labs.IDCLINICNUMBER = fac.LOCATIONID
    LEFT JOIN CW_GAP_PERIOD_PREVALENCE_VW per
    ON labs.IDPATIENT = per.ORGPATIENTID
    AND cln.IDCORPCLINIC = LPAD(CAST(per.ORGFACILITYCODE AS VARCHAR(5)), 5, '0')
    WHERE NUM_CLINICS > 1
    AND cln.AUD_CURRENT_ROW_FLAG = 'Y'
    AND fac.REC_CURRENT_IND = 1
    AND fac.REC_DELETED_IND <> 1
    ) X
  ) Y
WHERE ADMIT_ROW_NUM = 1
ORDER BY NUMBER_OF_CLINICS ASC, MPI, HOME_FACILITY DESC, FACILITY_NUMBER
    ```

# Pandas + SQL

## Using SQL-like queries in Pandas - `df.query()`

- Pandas DataFrames have a method called `.query()`
- This allows us to use SQL-like commands to reference data.
```python
## Normal Pandas Syntax
foo_df = bar_df.loc[bar_df['Col_1']>bar_df['Col_2']]
```

```python
## Using .query()
foo_df = bar_df.query("Col_1 > Col_2")
```
- How to use:
    - Enter the querty as a single string, using just column names to reference data.
    - To use and/or statements, use `&` and `|`, respectively

```python
foo_df = bar_df.query("Col_1 > Col_2 & Col_2 <= Col_3")
```

## Using SQL syntax with `pandasql`


- There is a library is called [pandasql](https://pypi.org/project/pandasql/) that allows for sql queries with pandas

We can install `pandasql` using the bash command `pip install pandasql`.

### Importing pandasql

- In order to use `pandasql`, we need to start by importing a `sqldf` object from `pandasql`
- You must also already have loaded in all dataframes that you wish to query
```python
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
```

### Writing Queries
```python
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meats m
     INNER JOIN
        births b
           ON m.date = b.date;"""

results = pysqldf(q)

```

## DATABASE ADMIN 101

### Creating Tables/SQL Data Types

- `CREATE TABLE IF NOT EXISTS table_name`
    -  Must include (col_name, datatype, and if its the key)
```PYTHON
  cur.execute("""CREATE TABLE IF NOT EXISTS cats (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    breed TEXT)
    """)  
    ```
    

- Data types in SQLite3:
    - https://www.sqlite.org/datatype3.html
    
- Data types:
    - TEXT
    - INTEGER
    - REAL
    - BLOB
    - NULL

#### Dropping a Table
- `DROP TABLE table_name`
    - `DROP TABLE IF EXISTS table_name`
    
    

#### Adding values to a table
- `INSERT INTO table_name`
    - list the columns to fill in first and then the VALUES()   
```python
cur.execute('''INSERT INTO cats (name, age, breed) 
                  VALUES ('Maru', 3, 'Scottish Fold');
            ''')
```

- To add multiple:
```python
cur.execute('''INSERT INTO cats (name, age, breed) 
            VALUES (?, ?, ?);
      ''',(dict_cats))
```
- `UPDATE`

```python
for dct in contacts:
    fname = dct['firstName']
    lname = dct['lastName']
    role = dct['role']
    phone = dct['telephone ']
    street = dct['street']
    city = dct['city']
    state = dct['state']
    z = dct['zipcode ']

    cur.execute('''INSERT INTO contactInfo (firstname, lastname, role, telephone, street, city, state, zipcode)
VALUES (?,?,?,?,?,?,?,?)''', (fname, lname, role, phone, street, city, state, z))
```

In [ ]:
# cur.execute("""CREATE TABLE cats (
#     id INTEGER PRIMARY KEY,
#     name TEXT,
#     age INTEGER,
#     breed TEXT)
#     """)

# 🏝 Breakout Group Activity:  [Survive on sql-island](https://sql-island.informatik.uni-kl.de/)

- Survive on sql-island
    - https://sql-island.informatik.uni-kl.de/
- 3 min walk through together before breakout rooms

#### Fist, change language to English
- click white menu button. 
<img src="https://raw.githubusercontent.com/flatiron-school/Online-DS-FT-022221-Cohort-Notes/master/assets/images/sql-island/SQL%20Island%201.png">

- Select option to change language (option 3)
<img src="https://raw.githubusercontent.com/flatiron-school/Online-DS-FT-022221-Cohort-Notes/master/assets/images/sql-island/SQL%20Island%202.png" width=35%>

- Select English
<img src="https://raw.githubusercontent.com/flatiron-school/Online-DS-FT-022221-Cohort-Notes/master/assets/images/sql-island/SQL%20Island%203.png" width=35%>

#### Take Note of the Tables and Column Names

<img src="https://raw.githubusercontent.com/flatiron-school/Online-DS-FT-022221-Cohort-Notes/master/assets/images/sql-island/SQL%20Island%204.png" width=80%>